# Import delle librerie utilizzate nel progetto

In [1]:
import os
from math import *
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date, datetime
import warnings
warnings.filterwarnings('ignore')

path = 'dataset.xlsx'

## Import delle classi e le funzioni costruite ad hoc per il progetto

In [2]:
# from classes import 
from functions import ammortamento_francese, ammortamento_italiano, ammortamento_bullet, market_data, estimation_rate_function, discount_rate_function

# Inserimento degli Input

In [3]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EURIBOR 3M'
except:
  underlying = 'EURIBOR 3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

try:
  rule_fixing = input('Inserire la regola per le fixing date: ')
  if rule_fixing == '':
     rule_fixing = '-2BD'
except:
  rule_fixing = '-2BD'

Inserire il sottostante:  
Inserire il piano di ammortamento:  
Inserire il nozionale:  100000000
Inserire il tasso annuo:  
Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni):  
Inserire la data iniziale del finanziamento in formato dd/mm/yyyy:  03/05/2021
Inserire il periodo che intercorre tra le rate in numero di mesi:  
Inserire la convenzione: 
Inserire la regola per il calcolo delle date: -1 mese
Inserire la regola per le fixing date:  


# Schedule delle date di pagamento <br> (viene utilizzato solamente il calendario TARGET in quanto ci si aspetta come sottostante solamente l'EURIBOR)

In [4]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor = Period(tenor, Months)
endofmonth = False
terminationDateConvention = business_convention
df = pd.DataFrame({'startdate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[:-1],'enddate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[1:]})

In [5]:
# Calcolo numero delle rate totali dell'ammortamento
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor) * int(maturity_label.rsplit(maturity_label[-1], 1)[0]))

# Creazione del piano di ammortamento a seconda di quello selezionato in input

In [6]:
if (amortization_type == 'Francese'):
  df_output = (ammortamento_francese(df,amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Italiano'):
  df_output = (ammortamento_italiano(df,amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Bullet'):
  df_output = (ammortamento_bullet(df,amortization_rate,tenor,rate_totali,notional))
print (df_output)

             startdate             enddate  Debito Residuo
0        May 5th, 2021    August 5th, 2021     100000000.0
1     August 5th, 2021  November 5th, 2021     100000000.0
2   November 5th, 2021  February 7th, 2022     100000000.0
3   February 7th, 2022       May 5th, 2022     100000000.0
4        May 5th, 2022    August 5th, 2022     100000000.0
5     August 5th, 2022  November 7th, 2022     100000000.0
6   November 7th, 2022  February 6th, 2023     100000000.0
7   February 6th, 2023       May 5th, 2023     100000000.0
8        May 5th, 2023    August 7th, 2023     100000000.0
9     August 7th, 2023  November 6th, 2023     100000000.0
10  November 6th, 2023  February 5th, 2024     100000000.0
11  February 5th, 2024       May 6th, 2024     100000000.0
12       May 6th, 2024    August 5th, 2024     100000000.0
13    August 5th, 2024  November 5th, 2024     100000000.0
14  November 5th, 2024  February 5th, 2025     100000000.0
15  February 5th, 2025       May 5th, 2025     100000000

# Calcolo delle Fixing Date utilizzate poi per la stima dei tassi forward

In [7]:
column_list = ['fixing date','fixing start','fixing end']
for col in column_list:
  df_output[col] = np.zeros

for i, date in enumerate(df_output['startdate']):
  if rule_fixing == '-2BD':
    date_fix = calendar.advance(date, -2, Days)
  else:
    date_fix = calendar.advance(date, -1, Months)
    date_fix = calendar.endOfMonth(date_fix)
    date_fix = calendar.advance(date_fix, -1, Days)

  df_output['fixing date'].iloc[i] = date_fix
  df_output['fixing start'].iloc[i] = calendar.advance(date_fix, 2, Days)
for t, date in enumerate(df_output['fixing start']):
  df_output['fixing end'].iloc[t] = calendar.advance(date, tenor, Months, ModifiedFollowing)

In [8]:
df_output

,startdate,enddate,Debito Residuo,fixing date,fixing start,fixing end
0,"May 5th, 2021","August 5th, 2021",100000000.0,"May 3rd, 2021","May 5th, 2021","August 5th, 2021"
1,"August 5th, 2021","November 5th, 2021",100000000.0,"August 3rd, 2021","August 5th, 2021","November 5th, 2021"
2,"November 5th, 2021","February 7th, 2022",100000000.0,"November 3rd, 2021","November 5th, 2021","February 7th, 2022"
3,"February 7th, 2022","May 5th, 2022",100000000.0,"February 3rd, 2022","February 7th, 2022","May 9th, 2022"
4,"May 5th, 2022","August 5th, 2022",100000000.0,"May 3rd, 2022","May 5th, 2022","August 5th, 2022"
5,"August 5th, 2022","November 7th, 2022",100000000.0,"August 3rd, 2022","August 5th, 2022","November 7th, 2022"
6,"November 7th, 2022","February 6th, 2023",100000000.0,"November 3rd, 2022","November 7th, 2022","February 7th, 2023"
7,"February 6th, 2023","May 5th, 2023",100000000.0,"February 2nd, 2023","February 6th, 2023","May 8th, 2023"
8,"May 5th, 2023","August 7th, 2023",100000000.0,"May 3rd, 2023","May 5th, 2023","August 7th, 2023"
9,"August 7th, 2023","November 6th, 2023",100000000.0,"August 3rd, 2023","August 7th, 2023","November 7th, 2023"


# Converto le date della curva importata in Date QuantLib

In [9]:
market_dict = market_data(path)
for i, d in enumerate(market_dict['ESTIMATION'].zero_curve_rate_df['DATE']):
  market_dict['ESTIMATION'].zero_curve_rate_df['DATE'].iloc[i] = Date(d.day,d.month,d.year)
for i, d in enumerate(market_dict['DISCOUNT'].zero_curve_rate_df['DATE']):
  market_dict['DISCOUNT'].zero_curve_rate_df['DATE'].iloc[i] = Date(d.day,d.month,d.year)

In [10]:
if underlying == 'EURIBOR 1W':
    Fixing_EUR = market_dict['HISTORICAL'].historical_fixed_rate_df['FIXING_VALUE'][0]
elif underlying == 'EURIBOR 1M':
    Fixing_EUR = market_dict['HISTORICAL'].historical_fixed_rate_df['FIXING_VALUE'][1]
elif underlying == 'EURIBOR 3M':
    Fixing_EUR = market_dict['HISTORICAL'].historical_fixed_rate_df['FIXING_VALUE'][2]
elif underlying == 'EURIBOR 6M':
    Fixing_EUR = market_dict['HISTORICAL'].historical_fixed_rate_df['FIXING_VALUE'][3]
elif underlying == 'EURIBOR 1Y':
    Fixing_EUR = market_dict['HISTORICAL'].historical_fixed_rate_df['FIXING_VALUE'][4]

# Chiamata della funzione per l'interpolazione dei forward rate

In [11]:
forward_rate = estimation_rate_function(market_dict['ESTIMATION'].zero_curve_rate_df,df_output['fixing start'], df_output['fixing end'],ref_date, Fixing_EUR)

# Chiamata della funzione per l'interpolazione dei discount rate

In [12]:
discount_factor = discount_rate_function(market_dict['DISCOUNT'].zero_curve_rate_df,ref_date,df_output)

In [13]:
df_output

,startdate,enddate,Debito Residuo,fixing date,fixing start,fixing end
0,"May 5th, 2021","August 5th, 2021",100000000.0,"May 3rd, 2021","May 5th, 2021","August 5th, 2021"
1,"August 5th, 2021","November 5th, 2021",100000000.0,"August 3rd, 2021","August 5th, 2021","November 5th, 2021"
2,"November 5th, 2021","February 7th, 2022",100000000.0,"November 3rd, 2021","November 5th, 2021","February 7th, 2022"
3,"February 7th, 2022","May 5th, 2022",100000000.0,"February 3rd, 2022","February 7th, 2022","May 9th, 2022"
4,"May 5th, 2022","August 5th, 2022",100000000.0,"May 3rd, 2022","May 5th, 2022","August 5th, 2022"
5,"August 5th, 2022","November 7th, 2022",100000000.0,"August 3rd, 2022","August 5th, 2022","November 7th, 2022"
6,"November 7th, 2022","February 6th, 2023",100000000.0,"November 3rd, 2022","November 7th, 2022","February 7th, 2023"
7,"February 6th, 2023","May 5th, 2023",100000000.0,"February 2nd, 2023","February 6th, 2023","May 8th, 2023"
8,"May 5th, 2023","August 7th, 2023",100000000.0,"May 3rd, 2023","May 5th, 2023","August 7th, 2023"
9,"August 7th, 2023","November 6th, 2023",100000000.0,"August 3rd, 2023","August 7th, 2023","November 7th, 2023"


## Calcolo dell'NPV della gamba variabile


In [1]:
year_fraction = (df_output['enddate'] - df_output['startdate'])/ 360
df_output['Rate'] = forward_rate
df_output['Flow'] = year_fraction * df_output['Debito Residuo'] * df_output['Rate']
df_output['discount factor'] = discount_factor
df_output['Discounted Flow'] = round((df_output['Flow'] * df_output['discount factor']),2)
NPV_leg_float = df_output['Discounted Flow'].sum()
NPV_leg_float 


NameError: name 'df_output' is not defined

## BPV 

In [15]:
BPV = year_fraction * df_output['Debito Residuo'] * 0.0001 * df_output['discount factor']
BPV.sum()

51396.143007810926

In [16]:
bp_margine = 0.0025
Utile = year_fraction * df_output['Debito Residuo'] * bp_margine
Utile.sum()

1268055.555555556

In [17]:
NPV_obiettivo = BPV.sum() + Utile.sum()

In [18]:
NPV_obiettivo 

1319451.698563367

In [19]:
fixed_leg = year_fraction * df_output['Debito Residuo'] * df_output['discount factor']

In [20]:
tasso_certo = (-NPV_obiettivo + NPV_leg_float)/ fixed_leg.sum()

In [21]:
tasso_certo

-0.0056996255499525954

In [22]:
df_output

,startdate,enddate,Debito Residuo,fixing date,fixing start,fixing end,Rate,Flow,discount factor,Discounted Flow
0,"May 5th, 2021","August 5th, 2021",100000000.0,"May 3rd, 2021","May 5th, 2021","August 5th, 2021",-0.005366,-137131.111111,1.001461,-137331.467954
1,"August 5th, 2021","November 5th, 2021",100000000.0,"August 3rd, 2021","August 5th, 2021","November 5th, 2021",-0.005350,-136722.222222,1.002941,-137124.352803
2,"November 5th, 2021","February 7th, 2022",100000000.0,"November 3rd, 2021","November 5th, 2021","February 7th, 2022",-0.005350,-139694.444444,1.004465,-140318.200473
3,"February 7th, 2022","May 5th, 2022",100000000.0,"February 3rd, 2022","February 7th, 2022","May 9th, 2022",-0.005300,-128083.3356,1.005880,-128836.445242
4,"May 5th, 2022","August 5th, 2022",100000000.0,"May 3rd, 2022","May 5th, 2022","August 5th, 2022",-0.005190,-132633.333333,1.007391,-133613.573144
5,"August 5th, 2022","November 7th, 2022",100000000.0,"August 3rd, 2022","August 5th, 2022","November 7th, 2022",-0.005046,-131765.920529,1.008917,-132940.912883
6,"November 7th, 2022","February 6th, 2023",100000000.0,"November 3rd, 2022","November 7th, 2022","February 7th, 2023",-0.004822,-121877.290018,1.010345,-123138.075371
7,"February 6th, 2023","May 5th, 2023",100000000.0,"February 2nd, 2023","February 6th, 2023","May 8th, 2023",-0.004437,-108462.972,1.011650,-109726.603432
8,"May 5th, 2023","August 7th, 2023",100000000.0,"May 3rd, 2023","May 5th, 2023","August 7th, 2023",-0.004083,-106608.158783,1.012942,-107987.830375
9,"August 7th, 2023","November 6th, 2023",100000000.0,"August 3rd, 2023","August 7th, 2023","November 7th, 2023",-0.003730,-94277.300373,1.014114,-95607.968331
